In [87]:
import os
import IPython.display as ipd
import numpy as np
import librosa, librosa.display
from scipy.fft import rfft
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

#### The raw training/validation/test data are the paths to the audio files, labels for now are just the emotions. The gender and intensity lists maybe further used as features

In [5]:
raw_train_data = []
raw_val_data = []
raw_test_data = []

train_label = []
val_label = []
test_label = []

train_gender = []
val_gender = []
test_gender = []

train_intensity = []
val_intensity = []
test_intensity = []

for i in range(0, 16):
    # Get all file names in the dir
    actor = "Actor_%02d" % (i + 1)
    inputs = os.listdir('./data/RAV/' + actor)
    
    for ele in inputs:
        raw_train_data.append('./data/RAV/' + actor + '/' + ele) # Form paths to the files
        
        file_name = ele.split('-')
        train_label.append(int(file_name[2]))                    # Get emotion label
        train_intensity.append(int(file_name[3]))                # Get intensity
        train_gender.append((i + 1) % 2)                         # Get gender (1 for male, 0 for female)
    
for i in range(16, 20):
    # Get all file names in the dir
    actor = "Actor_%02d" % (i + 1)
    inputs = os.listdir('./data/RAV/' + actor)

    for ele in inputs:
        raw_val_data.append('./data/RAV/' + actor + '/' + ele) # Form paths to the files
        
        file_name = ele.split('-')
        val_label.append(int(file_name[2]))                    # Get emotion label
        val_intensity.append(int(file_name[3]))                # Get intensity
        val_gender.append((i + 1) % 2)                         # Get gender (1 for male, 0 for female)
        
for i in range(20, 24):
    # Get all file names in the dir
    actor = "Actor_%02d" % (i + 1)
    inputs = os.listdir('./data/RAV/' + actor)

    for ele in inputs:
        raw_test_data.append('./data/RAV/' + actor + '/' + ele) # Form paths to the files
        
        file_name = ele.split('-')
        test_label.append(int(file_name[2]))                    # Get emotion label
        test_intensity.append(int(file_name[3]))                # Get intensity
        test_gender.append((i + 1) % 2)                         # Get gender (1 for male, 0 for female)

#### Use librosa to load the audio files

In [47]:
lbr_train_data = []
for ele in raw_train_data:
    temp, _ = librosa.load(ele)
    lbr_train_data.append(np.pad(temp, (0, 116247 - len(temp))))

In [48]:
lbr_val_data = []
for ele in raw_val_data:
    temp, _ = librosa.load(ele)
    lbr_val_data.append(np.pad(temp, (0, 116247 - len(temp))))

In [49]:
lbr_test_data = []
for ele in raw_test_data:
    temp, _ = librosa.load(ele)
    lbr_test_data.append(np.pad(temp, (0, 116247 - len(temp))))

In [46]:
np.pad([1,2,3], (0, 5))

array([1, 2, 3, 0, 0, 0, 0, 0])

#### Get the Fourier transform of the data as features
The result of from the Fourier transform on the entire data set may not offer much information since all information about time in the sound is inherently lost. In human speech, having a high pitch at the beginning may indicate very different emotions from having a high pitch at the end. I do not expect great results from such features.

In [84]:
fft_train_data = []
for ele in lbr_train_data:
    temp = rfft(ele)
    fft_train_data.append(temp.real)
        
fft_val_data = []
for ele in lbr_val_data:
    temp = rfft(ele)
    fft_val_data.append(temp.real)
        
fft_test_data = []
for ele in lbr_test_data:
    temp = rfft(ele)
    fft_test_data.append(temp.real)

#### Using KNN classifier for FFT features
As seen here, FFT+KNN yields validation errors slightly better than chance, I would expect similar results for the test error, therefore, I did not evaluate based on the test data.

In [102]:
for n in [1,3,5,10,15,20,30,40,50]:
    knn_classifier = KNeighborsClassifier(n_neighbors = n)
    knn_classifier.fit(fft_train_data, train_label)
    print("n = " + str(n) + ":" + str(knn_classifier.score(fft_val_data, val_label)))

n = 1:0.11666666666666667
n = 3:0.11666666666666667
n = 5:0.125
n = 10:0.1
n = 15:0.125
n = 20:0.10833333333333334
n = 30:0.14166666666666666
n = 40:0.13333333333333333
n = 50:0.13333333333333333


Due to the high dimensionality of the FFT features, PCA is applied to reduce the effect of the Curse of Dimensionality and improve performance.

In [107]:
for nc in [1,2,4,8,16,32,64,128,256,512]:
    pca = PCA(n_components = nc)
    fft_train_pca = pca.fit_transform(fft_train_data)
    fft_val_pca = pca.transform(fft_val_data)
    fft_test_pca = pca.transform(fft_test_data)
    print("n_components = " + str(nc))
    
    for n in [1,3,5,10,15,20,30,40,50]:
        knn_classifier = KNeighborsClassifier(n_neighbors = n)
        knn_classifier.fit(fft_train_pca, train_label)
        print("    n = " + str(n) + ":" + str(knn_classifier.score(fft_val_pca, val_label)))

n_components = 1
    n = 1:0.1625
    n = 3:0.17083333333333334
    n = 5:0.2
    n = 10:0.21666666666666667
    n = 15:0.175
    n = 20:0.2
    n = 30:0.19166666666666668
    n = 40:0.19583333333333333
    n = 50:0.19583333333333333
n_components = 2
    n = 1:0.15416666666666667
    n = 3:0.15833333333333333
    n = 5:0.19166666666666668
    n = 10:0.21666666666666667
    n = 15:0.2125
    n = 20:0.2125
    n = 30:0.19583333333333333
    n = 40:0.2
    n = 50:0.19166666666666668
n_components = 4
    n = 1:0.2
    n = 3:0.16666666666666666
    n = 5:0.2125
    n = 10:0.23333333333333334
    n = 15:0.225
    n = 20:0.23333333333333334
    n = 30:0.22916666666666666
    n = 40:0.2375
    n = 50:0.2375
n_components = 8
    n = 1:0.2375
    n = 3:0.2
    n = 5:0.20416666666666666
    n = 10:0.19583333333333333
    n = 15:0.21666666666666667
    n = 20:0.17916666666666667
    n = 30:0.21666666666666667
    n = 40:0.2375
    n = 50:0.23333333333333334
n_components = 16
    n = 1:0.1625
    n

In [115]:
# Find the test error for a good setting of hyper-parameters.
pca = PCA(n_components = 4)
fft_train_pca = pca.fit_transform(fft_train_data)
fft_val_pca = pca.transform(fft_val_data)
fft_test_pca = pca.transform(fft_test_data)

knn_classifier = KNeighborsClassifier(n_neighbors = 40)
knn_classifier.fit(fft_train_pca, train_label)

knn_classifier.score(fft_test_pca, test_label)

0.20416666666666666

## Good setting for KNN using FFT features with PCA: K = 30 / 40 / 50, n_components = 4 / 8. Validation error = 0.2375

#### Get spectrograms of the audio input as features as to preserve the time dimension of the input

In [73]:
stft_train_data = []
for ele in lbr_train_data:
    temp = librosa.stft(ele, n_fft = 512)
    stft_train_data.append(temp.real.reshape(-1))
        
stft_val_data = []
for ele in lbr_val_data:
    temp = librosa.stft(ele, n_fft = 512)
    stft_val_data.append(temp.real.reshape(-1))
        
stft_test_data = []
for ele in lbr_test_data:
    temp = librosa.stft(ele, n_fft = 512)
    stft_test_data.append(temp.real.reshape(-1))

#### Using KNN with spectrogram as features  
Due to the high number of variables in a spectrogram, making a prediction with this model is extremely slow. The validation error seems to be slightly better than using FFT features, perhaps I would need to shift the spectrograms in the training data set to get better performance.

In [74]:
# Training the knn classifier
knn_classifier = KNeighborsClassifier(n_neighbors = 30)
knn_classifier.fit(stft_train_data, train_label)

KNeighborsClassifier(n_neighbors=30)

In [81]:
for n in [1,3,5,10,15,20,30,40,50]:
    knn_classifier = KNeighborsClassifier(n_neighbors = n)
    knn_classifier.fit(stft_train_data, train_label)
    print("n = " + str(n) + ":" + str(knn_classifier.score(stft_val_data, val_label)))

1:0.1375
3:0.11666666666666667
5:0.12083333333333333
10:0.10416666666666667
15:0.125
20:0.10833333333333334
30:0.14166666666666666
40:0.13333333333333333
50:0.13333333333333333


In [112]:
for nc in [1,2,4,8,16,32,64,128,256,512]:
    pca = PCA(n_components = nc)
    stft_train_pca = pca.fit_transform(stft_train_data)
    stft_val_pca = pca.transform(stft_val_data)
    stft_test_pca = pca.transform(stft_test_data)
    print("n_components = " + str(nc))
    
    for n in [1,3,5,10,15,20,30,40,50]:
        knn_classifier = KNeighborsClassifier(n_neighbors = n)
        knn_classifier.fit(stft_train_pca, train_label)
        print("    n = " + str(n) + ":" + str(knn_classifier.score(stft_val_pca, val_label)))

n_components = 1
    n = 1:0.16666666666666666
    n = 3:0.16666666666666666
    n = 5:0.16666666666666666
    n = 10:0.20833333333333334
    n = 15:0.22083333333333333
    n = 20:0.225
    n = 30:0.19166666666666668
    n = 40:0.20416666666666666
    n = 50:0.225
n_components = 2
    n = 1:0.175
    n = 3:0.17916666666666667
    n = 5:0.19583333333333333
    n = 10:0.2125
    n = 15:0.23333333333333334
    n = 20:0.2125
    n = 30:0.22916666666666666
    n = 40:0.21666666666666667
    n = 50:0.20416666666666666
n_components = 4
    n = 1:0.2125
    n = 3:0.19166666666666668
    n = 5:0.19583333333333333
    n = 10:0.19166666666666668
    n = 15:0.22916666666666666
    n = 20:0.225
    n = 30:0.225
    n = 40:0.2375
    n = 50:0.2375
n_components = 8
    n = 1:0.17083333333333334
    n = 3:0.1625
    n = 5:0.22083333333333333
    n = 10:0.20416666666666666
    n = 15:0.225
    n = 20:0.2125
    n = 30:0.22916666666666666
    n = 40:0.23333333333333334
    n = 50:0.23333333333333334
n_c

In [114]:
# Find the test error for a good setting of hyper-parameters.
pca = PCA(n_components = 4)
stft_train_pca = pca.fit_transform(stft_train_data)
stft_val_pca = pca.transform(stft_val_data)
stft_test_pca = pca.transform(stft_test_data)

knn_classifier = KNeighborsClassifier(n_neighbors = 40)
knn_classifier.fit(stft_train_pca, train_label)

knn_classifier.score(stft_test_pca, test_label)

0.24583333333333332

## Good setting of hyper-parameters for Spectrogram features with KNN and PCA for dimension reduction: K = 40 / 50, n_components = 4. Validation accuracy = 0.2375, test accuracy = 0.24583